<a href="https://colab.research.google.com/github/MarceloEscobal/MarceloEscobal/blob/main/TP1/Entrenamiento_de_modelo_CORPUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Carga del CSV y Preprocesamiento de texto

In [6]:

import pandas as pd
import re
from collections import Counter

# 1. Cargar el CSV
df = pd.read_csv("correos.csv")

# 2. Limpiar texto
def limpiar_texto(texto):
    if pd.isna(texto):
        return ""
    texto = texto.lower()
    texto = re.sub(r"\d+", "", texto)  # Quitar números
    texto = re.sub(r"[^\w\s]", "", texto)  # Quitar signos
    texto = texto.strip()
    return texto

df["contenido_limpio"] = df["contenido"].apply(limpiar_texto)

# 3. Tokenización básica sin NLTK
df["tokens"] = df["contenido_limpio"].apply(lambda x: x.split())

# 4. Contar todas las palabras del corpus
todas_palabras = [palabra for lista in df["tokens"] for palabra in lista]
frecuencias = Counter(todas_palabras)

print("🔹 Palabras más frecuentes:")
for palabra, freq in frecuencias.most_common(30):
    print(f"{palabra}: {freq}")


🔹 Palabras más frecuentes:
de: 476
n: 250
orden: 217
formulario: 194
pago: 126
tributo: 90
electronica: 69
planilla: 59
notificación: 55
asunto: 52
pdt: 39
igvrenta: 35
mensualiev: 35
sol: 33
en: 32
buzón: 32
electrónico: 32
valores: 29
resoluciones: 24
del: 22
coactiva: 20
y: 20
resolución: 19
no: 19
cobranza: 17
e: 17
emisión: 13
comprobantes: 12
plataforma: 12
confirmación: 12


## 2. Clasificación básica con diccionario manual

In [7]:

diccionario = {
    "deuda": ["coactiva", "retencion", "requerimiento", "pago", "adeudo", "monto"],
    "comprobante": ["factura", "boleta", "recibo", "documento"],
    "citacion": ["citacion", "comparecer", "esquela", "presente"]
}

def clasificar_correo(texto, diccionario):
    for categoria, palabras in diccionario.items():
        for palabra in palabras:
            if palabra in texto:
                return categoria
    return "no_clasificado"

df["categoria"] = df["contenido_limpio"].apply(lambda x: clasificar_correo(x, diccionario))
df[["id", "titulo", "fecha", "categoria"]].to_csv("correos_clasificados.csv", index=False)


## 3. Entrenamiento supervisado con Logistic Regression

In [8]:

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from nltk.corpus import stopwords

nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')

train = pd.read_csv("train_con_contenido_mejorado.csv")
test = pd.read_csv("test_con_contenido.csv")

X_train = train["contenido"].fillna("")
y_train = train["prioridad"]
X_test = test["contenido"].fillna("")
y_test = test["prioridad"]

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=spanish_stopwords)),
    ('clf', LogisticRegression(class_weight='balanced', max_iter=1000))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("📊 Reporte de Clasificación:\n")
print(classification_report(y_test, y_pred))
print("🧱 Matriz de Confusión:\n")
print(confusion_matrix(y_test, y_pred))


📊 Reporte de Clasificación:

                 precision    recall  f1-score   support

 1 - importante       0.96      1.00      0.98        47
2 - informativo       1.00      0.85      0.92        13

       accuracy                           0.97        60
      macro avg       0.98      0.92      0.95        60
   weighted avg       0.97      0.97      0.97        60

🧱 Matriz de Confusión:

[[47  0]
 [ 2 11]]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 4. Refuerzo del entrenamiento con nuevos correos informativos

In [9]:

nuevos_informativos = [
    "Estimado usuario, nuestras oficinas estarán cerradas este viernes por mantenimiento.",
    "Le informamos que la plataforma ha sido actualizada para mejorar su experiencia.",
    "Este mensaje es solo para confirmar que su sesión se ha cerrado correctamente.",
    "A partir del próximo mes, nuestros horarios de atención cambiarán.",
    "Hemos actualizado nuestra política de privacidad. Puede revisarla en el portal web.",
    "Gracias por su preferencia. Este correo no requiere ninguna acción.",
    "Nos encontramos en proceso de mejora de servicios. No afecta su cuenta.",
    "Se ha registrado correctamente su dirección de correo electrónico.",
    "Recuerde que puede acceder a su historial desde la sección 'Mi cuenta'.",
    "Este mensaje es informativo. No contiene deudas ni vencimientos pendientes."
]

df_nuevos_info = pd.DataFrame({
    "id": range(1000, 1000 + len(nuevos_informativos)),
    "titulo": ["[INFO]"] * len(nuevos_informativos),
    "fecha": ["2025-05-06"] * len(nuevos_informativos),
    "prioridad": ["2 - informativo"] * len(nuevos_informativos),
    "contenido": nuevos_informativos
})

df_train = pd.read_csv("train_con_contenido_mejorado.csv")
df_train_expandidos = pd.concat([df_train, df_nuevos_info], ignore_index=True)
df_train_expandidos.to_csv("train_con_contenido_reforzado.csv", index=False)


## 5. Función de predicción y pruebas con nuevos correos

In [10]:

import joblib
def predecir_prioridad(correo_texto: str) -> str:
    if not correo_texto.strip():
        return "Texto vacío"
    texto_df = pd.Series([correo_texto])
    prediccion = pipeline.predict(texto_df)[0]
    return prediccion

correos_prueba = {
    "Deuda coactiva": "Estimado contribuyente, se ha iniciado un proceso de ejecución coactiva por deuda tributaria.",
    "Comprobante disponible": "El comprobante de pago electrónico ya está disponible.",
    "Pago próximo a vencer": "Recuerde que el pago del tributo mensual vence el 15.",
    "Correo informativo general": "Nuestros sistemas estarán en mantenimiento el fin de semana.",
    "Actualización de datos": "Actualice sus datos. Esta actualización no implica pagos.",
    "Notificación sin contexto claro": "Tiene un documento pendiente. Ingrese con su clave SOL.",
    "Boleta disponible": "La boleta electrónica ha sido generada y está en el sistema.",
    "Resolución sin palabras clave explícitas": "Adjuntamos la resolución del expediente N° 2025.",
    "Mensaje vacío": ""
}

for titulo, contenido in correos_prueba.items():
    print(f"🟢 {titulo}: {predecir_prioridad(contenido)}")


🟢 Deuda coactiva: 1 - importante
🟢 Comprobante disponible: 1 - importante
🟢 Pago próximo a vencer: 1 - importante
🟢 Correo informativo general: 2 - informativo
🟢 Actualización de datos: 2 - informativo
🟢 Notificación sin contexto claro: 2 - informativo
🟢 Boleta disponible: 2 - informativo
🟢 Resolución sin palabras clave explícitas: 2 - informativo
🟢 Mensaje vacío: Texto vacío
